In [114]:
import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [158]:
CLIENT_ID = 'RNQZEFS4EHVDI0CQERFZEHSLY3QYMBBNCRSSNUBGMFKIJBLB' 
CLIENT_SECRET = 'X2DFDPKNEA4TXS2SRMDJQ42IPBTZS45KQ2KYS112LTYHHJF2' 
VERSION = '20180605'
LIMIT = 100
CAT_ID = '4bf58dd8d48988d18d941735' # Foursquare category ID for gaming cafés

Obtaining board game cafés within Tampere.

In [281]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&categoryId={}&near={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    CAT_ID,
    'Tampere',
    LIMIT)

In [282]:
results = requests.get(url).json()

In [40]:
def get_category(row):
    categories_list = row['categories']
    if len(categories_list) == 0:
        return None
    return categories_list[0]['name']

In [172]:
[get_category(results['response']['venues'][n]) for n in range(len(results['response']['venues']))]

['Coffee Shop',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Gaming Cafe',
 'Casino']

In [283]:
results_pd = pd.json_normalize(results['response']['venues'])
results_pd = results_pd[results_pd['categories'].notnull()]
columns = ['name', 'categories', 'location.lat', 'location.lng', 'location.postalCode', 'location.city']
results_pd = results_pd[columns]
results_pd.head()

,name,categories,location.lat,location.lng,location.postalCode,location.city
0,Lautapelikahvila Taverna,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",61.499885,23.771130,33100,Tampere
1,Kaukisen jyystöluola,"[{'id': '4bf58dd8d48988d18d941735', 'name': 'G...",61.515075,23.620037,NaN,NaN
2,Nuorisokahvila Uniikki,"[{'id': '4bf58dd8d48988d18d941735', 'name': 'G...",61.489525,23.920553,NaN,Tampere
3,Nuorisotila Nasta,"[{'id': '4bf58dd8d48988d18d941735', 'name': 'G...",61.463335,23.646317,NaN,Pirkkala
4,Veetin Sänky,"[{'id': '4bf58dd8d48988d18d941735', 'name': 'G...",61.432522,24.126527,NaN,NaN


In [284]:
cat_list = list(results_pd['categories'].explode())
results_pd['category'] = pd.DataFrame(cat_list)['name']
results_pd.drop(['categories'], axis=1, inplace=True)
results_pd.head()

,name,location.lat,location.lng,location.postalCode,location.city,category
0,Lautapelikahvila Taverna,61.499885,23.771130,33100,Tampere,Coffee Shop
1,Kaukisen jyystöluola,61.515075,23.620037,NaN,NaN,Gaming Cafe
2,Nuorisokahvila Uniikki,61.489525,23.920553,NaN,Tampere,Gaming Cafe
3,Nuorisotila Nasta,61.463335,23.646317,NaN,Pirkkala,Gaming Cafe
4,Veetin Sänky,61.432522,24.126527,NaN,NaN,Gaming Cafe


In [285]:
rename_dict = {'location.lat': 'latitude', 
               'location.lng': 'longitude', 
               'location.postalCode': 'postalCode',
               'location.city': 'city'}
results_pd.rename(columns=rename_dict, inplace=True)
results_pd.head()

,name,latitude,longitude,postalCode,city,category
0,Lautapelikahvila Taverna,61.499885,23.771130,33100,Tampere,Coffee Shop
1,Kaukisen jyystöluola,61.515075,23.620037,NaN,NaN,Gaming Cafe
2,Nuorisokahvila Uniikki,61.489525,23.920553,NaN,Tampere,Gaming Cafe
3,Nuorisotila Nasta,61.463335,23.646317,NaN,Pirkkala,Gaming Cafe
4,Veetin Sänky,61.432522,24.126527,NaN,NaN,Gaming Cafe


In [204]:
results_pd

,name,latitude,longitude,postalCode,city,category
0,Lautapelikahvila Taverna,61.499885,23.771130,33100,Tampere,Coffee Shop
1,Kaukisen jyystöluola,61.515075,23.620037,NaN,NaN,Gaming Cafe
2,Nuorisokahvila Uniikki,61.489525,23.920553,NaN,Tampere,Gaming Cafe
3,Nuorisotila Nasta,61.463335,23.646317,NaN,Pirkkala,Gaming Cafe
4,Mansikkaveturi,61.598492,23.987669,NaN,NaN,Gaming Cafe
5,Jessen peliluola,61.468800,23.635300,NaN,NaN,Gaming Cafe
6,Lani luola,61.473180,23.363810,NaN,NaN,Gaming Cafe
7,iPad,61.477558,24.042317,NaN,Tää yks,Gaming Cafe
8,Veetin Sänky,61.432522,24.126527,NaN,NaN,Gaming Cafe
9,Kahvio,61.462196,23.642667,NaN,NaN,Gaming Cafe


In [286]:
tampere_pd = results_pd.copy()

In [287]:
tampere_pd = tampere_pd[ 
    tampere_pd['city'].eq('Tampere') & 
    tampere_pd['category'].isin(['Coffee Shop', 'Gaming Cafe'])]
tampere_pd = tampere_pd[tampere_pd['postalCode'].notna()]
tampere_pd.reset_index(inplace=True, drop=True)
tampere_pd

,name,latitude,longitude,postalCode,city,category
0,Lautapelikahvila Taverna,61.499885,23.771130,33100,Tampere,Coffee Shop
1,Miken peliluola,61.507538,23.691294,33270,Tampere,Gaming Cafe


Since Miken peliluola does not provide a result with the [Finnish Business Information System (YTJ)](https://www.ytj.fi/en/), we shall drop this result.

In [288]:
tampere_pd.drop(1, axis=0, inplace=True)
tampere_pd

,name,latitude,longitude,postalCode,city,category
0,Lautapelikahvila Taverna,61.499885,23.77113,33100,Tampere,Coffee Shop


In [289]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&categoryId={}&near={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    CAT_ID,
    'Helsinki',
    LIMIT)

In [290]:
results = requests.get(url).json()

In [291]:
results_pd = pd.json_normalize(results['response']['venues'])
results_pd = results_pd[results_pd['categories'].notnull()]
columns = ['name', 'categories', 'location.lat', 'location.lng', 'location.postalCode', 'location.city']
results_pd = results_pd[columns]
cat_list = list(results_pd['categories'].explode())
results_pd['category'] = pd.DataFrame(cat_list)['name']
results_pd.drop(['categories'], axis=1, inplace=True)
rename_dict = {'location.lat': 'latitude', 
               'location.lng': 'longitude',
               'location.postalCode': 'postalCode',
               'location.city': 'city'}
results_pd.rename(columns=rename_dict, inplace=True)
results_pd.head()

,name,latitude,longitude,postalCode,city,category
0,Fantasiapelit,60.169461,24.940527,00100,Helsinki,Hobby Shop
1,Café Boardgame,60.166878,24.935020,00100,Helsinki,Gaming Cafe
2,Fiasco,60.166706,24.921715,00180,Helsinki,Gaming Cafe
3,360XP,60.159943,24.943606,00150,Helsinki,Gaming Cafe
4,8-Bit Taproom,60.165988,24.937194,00120,Helsinki,Beer Bar


In [292]:
helsinki_pd = results_pd.copy()
helsinki_pd = helsinki_pd[ 
    helsinki_pd['city'].eq('Helsinki') & 
    helsinki_pd['category'].isin(['Coffee Shop', 'Gaming Cafe'])]
helsinki_pd = helsinki_pd[helsinki_pd['postalCode'].notna()]
helsinki_pd.reset_index(inplace=True, drop=True)
helsinki_pd

,name,latitude,longitude,postalCode,city,category
0,Café Boardgame,60.166878,24.935020,00100,Helsinki,Gaming Cafe
1,Fiasco,60.166706,24.921715,00180,Helsinki,Gaming Cafe
2,360XP,60.159943,24.943606,00150,Helsinki,Gaming Cafe
3,Angry Birds SM,60.169476,24.930865,00100,Helsinki,Gaming Cafe
4,Pelikioski,60.171260,24.940524,00100,Helsinki,Gaming Cafe


Checking YTJ shows that Fiasco, Angry Birds SM and Pelikioski are again not registered businesses. In addition, 360XP is a VR gaming experience, not a board game café and so we drop this too.

In [296]:
helsinki_pd.drop(list(range(1, len(helsinki_pd))), axis=0, inplace=True)
helsinki_pd

,name,latitude,longitude,postalCode,city,category
0,Café Boardgame,60.166878,24.93502,00100,Helsinki,Gaming Cafe


In [297]:
results_pd

,name,latitude,longitude,postalCode,city,category
0,Fantasiapelit,60.169461,24.940527,00100,Helsinki,Hobby Shop
1,Café Boardgame,60.166878,24.935020,00100,Helsinki,Gaming Cafe
2,Fiasco,60.166706,24.921715,00180,Helsinki,Gaming Cafe
3,360XP,60.159943,24.943606,00150,Helsinki,Gaming Cafe
4,8-Bit Taproom,60.165988,24.937194,00120,Helsinki,Beer Bar
5,Utopos Lounge,60.169112,24.936815,00100,Helsinki,Other Event
6,Angry Birds SM,60.169476,24.930865,00100,Helsinki,Gaming Cafe
7,Ompun Leikkipaikka,60.161083,24.740673,NaN,NaN,Gaming Cafe
8,Огни ночного Гонконга,60.163044,24.911768,00180,Helsinki,Movie Theater
9,Mindfield,60.179508,24.957901,NaN,NaN,Gaming Cafe


In [298]:
city_pds = dict()

In [ ]:
def get